# File management

The goal here is to merge all variables for their respective case using ```cdo mergetime```.

# Setup

In [30]:
import os
import shutil
import glob
import pandas as pd
import numpy as np
import xarray as xr
import pickle as pkl

import cartopy
import cartopy.crs as ccrs
import cartopy.mpl.ticker as cticker
from cartopy.util import add_cyclic_point

import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
import matplotlib.path as mpath

import cmocean.cm as cmo

import warnings
warnings.filterwarnings("ignore", message="Unable to decode time axis into full numpy.datetime64 objects, continuing using cftime.datetime objects instead, reason: dates out of range")
warnings.filterwarnings("ignore", message="Ambiguous reference date string: 101-01-01. The first value is assumed to be the year hence will be padded with zeros to remove the ambiguity (the padded reference date string is: 0101-01-01). To remove this message, remove the ambiguity by padding your reference date strings with zeros.")

In [2]:
## some magic to automatically reload my functions before running a new cell
# %load_ext autoreload
## %reload_ext autoreload
# %autoreload 1
# %aimport itcz_phys.cmiptools

# import itcz_phys.cmiptools as cmip

## Figure settings

In [3]:
import matplotlib as mpl
mpl.rcParams['figure.dpi'] = 120

# %config InlineBackend.figure_formats = ['pdf']
%config InlineBackend.figure_formats = ['png']

## Create constants

In [4]:
# Cases / experiments
cases = ['piControl', '1pctCO2', '1pctCO2-rad', '1pctCO2-bgc']
cases_rad = ['1pctCO2', '1pctCO2-rad']


# Models with data in /tiger/scratch/gpfs/GEOCLIM/bgb2/CMIP/
# --> removed MRI-ESM2-0 because 1pctCO2 and 1pctCO2-rad Amon variables are the same

# --> models with HT, VEG for 1pctCO2 and 1pctCO2-rad (i.e., the max number of models used)
models = ['ACCESS-ESM1-5', 'BCC-CSM2-MR', 'CESM2', 'CMCC-ESM2',
          'CNRM-ESM2-1', 'EC-Earth3-CC', 'GFDL-ESM4', 'GISS-E2-1-G',
          'IPSL-CM6A-LR', 'MIROC-ES2L', 'MPI-ESM1-2-LR', 'NorESM2-LM',
          'UKESM1-0-LL']
#  --> models with HT, VEG, SW for all cases
models_all = ['ACCESS-ESM1-5', 'BCC-CSM2-MR', 'CMCC-ESM2',
              'CNRM-ESM2-1', 'GISS-E2-1-G', 'IPSL-CM6A-LR', 'MIROC-ES2L',
              'MPI-ESM1-2-LR', 'UKESM1-0-LL']
# --> only models with HT, VEG, SW for 1pctCO2 and 1pctCO2-rad
models_radsw = ['EC-Earth3-CC']
# --> only models with HT, VEG for 1pctCO2 and 1pctCO2-rad
models_rad = ['CESM2', 'GFDL-ESM4', 'NorESM2-LM']


# Variables
variables = ['evspsbl', 'hfls', 'hfss', 'lai', 'pr', 'prsn', 'ps', 'rlds', 
             'rlus', 'rlut', 'rsds', 'rsdt', 'rsus', 'rsut', 'ta', 'tas',
             'uas', 'vas']
variables_ht = ['hfls', 'hfss', 'pr', 'prsn', 'ps', 'rlds', 'rlus', 'rlut',
                'rsds', 'rsdt', 'rsus', 'rsut', 'ta']
variables_vegsw = ['evspsbl', 'lai', 'tas', 'uas', 'vas']
variables_veg = ['evspsbl', 'lai', 'tas']


# Table ID
table_id = {
    'evspsbl': 'Amon',
    'hfls': 'Amon',
    'hfss': 'Amon',
    'lai': 'Lmon',
    'pr': 'Amon',
    'prsn': 'Amon',
    'ps': 'Amon',
    'rlds': 'Amon', 
    'rlus': 'Amon',
    'rlut': 'Amon',
    'rsds': 'Amon',
    'rsdt': 'Amon',
    'rsus': 'Amon',
    'rsut': 'Amon',
    'ta': 'Amon',
    'tas': 'Amon',
    'uas': 'Amon',
    'vas': 'Amon'
}


# Grid label
grid_label = {
    'ACCESS-ESM1-5': 'gn',
    'BCC-CSM2-MR': 'gn',
    'CESM2': 'gn',
    'CMCC-ESM2': 'gn',
    'CNRM-ESM2-1': 'gr',
    'EC-Earth3-CC': 'gr',
    'GFDL-ESM4': 'gr1',
    'GISS-E2-1-G': 'gn',
    'IPSL-CM6A-LR': 'gr',
    'MIROC-ES2L': 'gn',
    'MPI-ESM1-2-LR': 'gn',
    'MRI-ESM2-0': 'gn',
    'NorESM2-LM': 'gn',
    'UKESM1-0-LL': 'gn'
}


# Months
months = ['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec']
months_abbr = ['J','F','M','A','M','J','J','A','S','O','N','D']


# Directory where CMIP6 model output lives on tiger
cmipdir = '/tiger/scratch/gpfs/GEOCLIM/bgb2/CMIP/'
cmipmergedir = '/tiger/scratch/gpfs/GEOCLIM/bgb2/CMIPmerge/'

## Get helper dictionaries

### Create dictionaries

#### Variant ID

In [6]:
variant_id = {
    'ACCESS-ESM1-5': 'r1i1p1f1',
    'BCC-CSM2-MR': 'r1i1p1f1',
    'CanESM5': 'r1i1p1f1',
    'CESM2': 'r1i1p1f1',
    'CMCC-ESM2': 'r1i1p1f1',
    'CNRM-ESM2-1': 'r1i1p1f2',
    'EC-Earth3-CC': 'r1i1p1f1',
    'GFDL-ESM4': 'r1i1p1f1',
    'GISS-E2-1-G': 'r101i1p1f1',
    'IPSL-CM6A-LR': 'r1i1p1f1',
    'MIROC-ES2L': 'r1i1p1f2',
    'MPI-ESM1-2-LR': 'r1i1p1f1',
    'MRI-ESM2-0': 'r1i2p1f1',
    'NorESM2-LM': 'r1i1p1f1',
    'UKESM1-0-LL': 'r1i1p1f2'
}

# with open('../pkl_files/variant_id.pkl', 'wb') as file:
#     pkl.dump(variant_id, file, pkl.HIGHEST_PROTOCOL)

#### Branch times in PI

In [7]:
# Originally from an NCL file created by Lily Hahn
# for GISS-ES2-1-G with variant_id=r101i1p1f1 --> branch time is 185001-201412 (originally 415001-430012 in Lily's file)
branch_times_in_pi = {
  'ACCESS-CM2': '095001-144912', 'ACCESS-ESM1-5': '010101-060012', 'AWI-CM-1-1-MR': '265001-280012', 'BCC-CSM2-MR': '185001-244912', 'BCC-ESM1': '185001-230012',
  'CAMS-CSM1-0': '303001-339912', 'CESM2': '050101-069912', 'CESM2-FV2': '032101-050012', 'CESM2-WACCM': '007001-029912', 'CESM2-WACCM-FV2': '030101-049912',
  'CMCC-CM2-SR5': '185001-209912', 'CMCC-ESM2': '185001-209912', 'CNRM-CM6-1': '185001-234912', 'CNRM-CM6-1-HR': '185001-214912', 'CNRM-ESM2-1': '185001-234912',
  'CanESM5': '520101-540012', 'E3SM-1-0': '010101-025012', 'EC-Earth3': '254001-270012', 'EC-Earth3-AerChem': '185001-199912', 'EC-Earth3-Veg': '185001-200012',
  'FGOALS-f3-L': '060001-116012', 'FGOALS-g3': '046301-061912', 'GFDL-CM4': '025101-045012', 'GFDL-ESM4': '010101-030012', 'GISS-E2-1-G': '185001-201412',
  'GISS-E2-1-H': '318001-333012', 'GISS-E2-2-G': '200001-215012', 'HadGEM3-GC31-LL': '185001-204912', 'HadGEM3-GC31-MM': '185001-200912', 'IITM-ESM': '192601-212512',
  'INM-CM4-8': '194701-214912', 'INM-CM5-0': '209901-226812', 'IPSL-CM6A-LR': '187001-234912', 'MIROC-ES2L': '185001-204912', 'MIROC6': '320001-339912',
  'MPI-ESM-1-2-HAM': '195001-210912', 'MPI-ESM1-2-HR': '185001-199912', 'MPI-ESM1-2-LR': '185001-200912', 'MRI-ESM2-0': '185001-255012', 'NorCPM1': '025101-050012',
  'NorESM2-LM': '160001-210012', 'NorESM2-MM': '120001-134912', 'SAM0-UNICON': '027401-043012', 'TaiESM1': '020101-040012', 'UKESM1-0-LL': '196001-214912'
}

branch_start_year_in_pi = {}
for m in branch_times_in_pi.keys():
    raw_date = branch_times_in_pi[m][:6]
    branch_start_year_in_pi[m] = raw_date[:4]+'-'+raw_date[4:]

# l40 = last 40 years of 150 year 1% CO2 increase period
# start year
period_l40_start_year_in_pi = {}
for m in branch_times_in_pi.keys():
    raw_date = branch_times_in_pi[m][:6]
    len_date = len(raw_date)
    end_year = int(raw_date[:len_date-2])+100
    period_l40_start_year_in_pi[m] = str(end_year).zfill(4)+'01'
    
# end year
period_l40_end_year_in_pi = {}
for m in branch_times_in_pi.keys():
    raw_date = branch_times_in_pi[m][:6]
    len_date = len(raw_date)
    end_year = int(raw_date[:len_date-2])+139
    period_l40_end_year_in_pi[m] = str(end_year).zfill(4)+'12'

# slice
period_l40_slice_in_pi = {}
for m in branch_times_in_pi.keys():
    period_l40_slice_in_pi[m] = slice(period_l40_start_year_in_pi[m], period_l40_end_year_in_pi[m])

    
# with open('../pkl_files/branch_start_year_in_pi.pkl', 'wb') as file:
#     pkl.dump(branch_start_year_in_pi, file, pkl.HIGHEST_PROTOCOL)
# with open('../pkl_files/period_l40_slice_in_pi.pkl', 'wb') as file:
#     pkl.dump(period_l40_slice_in_pi, file, pkl.HIGHEST_PROTOCOL)
# with open('../pkl_files/period_l40_start_year_in_pi.pkl', 'wb') as file:
#     pkl.dump(period_l40_start_year_in_pi, file, pkl.HIGHEST_PROTOCOL)
# with open('../pkl_files/period_l40_end_year_in_pi.pkl', 'wb') as file:
#     pkl.dump(period_l40_end_year_in_pi, file, pkl.HIGHEST_PROTOCOL)

#### Times in CO2

In [10]:
# Get last 40 years of 1% CO2 period
period_l40_start_year_in_co2 = {}
period_l40_end_year_in_co2 = {}
period_l40_slice_in_co2 = {}

v = 'evspsbl'
for m in models:
    print(m)
    # for c in ['1pctCO2','1pctCO2-rad','1pctCO2-bgc']:
    for c in cases:
        print('  ',c)
        try:
            data = xr.open_dataset(cmipmergedir+m+'/'+v+'_'+table_id[v]+'_'+m+'_'+c+'_'+variant_id[m]+'_'+grid_label[m]+'.nc')

            # period_l40_start_year_in_co2[m] = str(data['time'].isel(time=0).values)[:4]
            # period_l40_start_year_in_co2[m] = str(int(period_l40_start_year_in_co2[m])+100).zfill(4)+'01'
            # period_l40_end_year_in_co2[m] = str(data['time'].isel(time=0).values)[:4]
            # period_l40_end_year_in_co2[m] = str(int(period_l40_end_year_in_co2[m])+139).zfill(4)+'12'
            # period_l40_slice_in_co2[m] = slice(period_l40_start_year_in_co2[m], period_l40_end_year_in_co2[m])

            print('    ',data['time'][0].values)
            print('    ',data['time'][-1].values)
            # print('    ',period_l40_start_year_in_co2[m])
            # print('    ',period_l40_end_year_in_co2[m])          

        except OSError:
            print('     ###')
    
# with open('../pkl_files/period_l40_slice_in_co2.pkl', 'wb') as file:
#     pkl.dump(period_l40_slice_in_co2, file, pkl.HIGHEST_PROTOCOL)
# with open('../pkl_files/period_l40_start_year_in_co2.pkl', 'wb') as file:
#     pkl.dump(period_l40_start_year_in_co2, file, pkl.HIGHEST_PROTOCOL)
# with open('../pkl_files/period_l40_end_year_in_co2.pkl', 'wb') as file:
#     pkl.dump(period_l40_end_year_in_co2, file, pkl.HIGHEST_PROTOCOL)

ACCESS-ESM1-5
   piControl
     0101-01-16 12:00:00
     1100-12-16 12:00:00
   1pctCO2
     0101-01-16 12:00:00
     0250-12-16 12:00:00
   1pctCO2-rad
     0101-01-16 12:00:00
     0250-12-16 12:00:00
   1pctCO2-bgc
     0101-01-16 12:00:00
     0250-12-16 12:00:00
BCC-CSM2-MR
   piControl
     ###
   1pctCO2
     ###
   1pctCO2-rad
     ###
   1pctCO2-bgc
     ###
CESM2
   piControl
     ###
   1pctCO2
     0001-01-15 12:00:00
     0150-12-15 12:00:00
   1pctCO2-rad
     0001-01-15 12:00:00
     0150-12-15 12:00:00
   1pctCO2-bgc
     ###
CMCC-ESM2
   piControl
     1850-01-16 12:00:00
     2349-12-16 12:00:00
   1pctCO2
     1850-01-16 12:00:00
     2014-12-16 12:00:00
   1pctCO2-rad


/home/bgb2/miniconda3/envs/itcz_phys/lib/python3.11/site-packages/xarray/coding/times.py:152: SerializationWarning: Ambiguous reference date string: 101-01-01. The first value is assumed to be the year hence will be padded with zeros to remove the ambiguity (the padded reference date string is: 0101-01-01). To remove this message, remove the ambiguity by padding your reference date strings with zeros.
  warnings.warn(warning_msg, SerializationWarning)
/home/bgb2/miniconda3/envs/itcz_phys/lib/python3.11/site-packages/xarray/coding/times.py:152: SerializationWarning: Ambiguous reference date string: 101-01-01. The first value is assumed to be the year hence will be padded with zeros to remove the ambiguity (the padded reference date string is: 0101-01-01). To remove this message, remove the ambiguity by padding your reference date strings with zeros.
  warnings.warn(warning_msg, SerializationWarning)
/home/bgb2/miniconda3/envs/itcz_phys/lib/python3.11/site-packages/xarray/coding/times.py

     1850-01-16 12:00:00
     2014-12-16 12:00:00
   1pctCO2-bgc
     1850-01-16 12:00:00
     2014-12-16 12:00:00
CNRM-ESM2-1
   piControl
     1850-01-16 12:00:00
     2349-12-16 12:00:00
   1pctCO2
     1850-01-16T12:00:00.000000000
     1999-12-16T12:00:00.000000000
   1pctCO2-rad
     1850-01-16T12:00:00.000000000
     1989-12-16T12:00:00.000000000
   1pctCO2-bgc
     1850-01-16T12:00:00.000000000
     1989-12-16T12:00:00.000000000
EC-Earth3-CC
   piControl
     ###
   1pctCO2
     1850-01-16T12:00:00.000000000
     2014-12-16T12:00:00.000000000
   1pctCO2-rad
     1850-01-16T12:00:00.000000000
     2014-12-16T12:00:00.000000000
   1pctCO2-bgc
     ###
GFDL-ESM4
   piControl
     ###
   1pctCO2
     0001-01-16 12:00:00
     0150-12-16 12:00:00
   1pctCO2-rad
     0001-01-16 12:00:00
     0150-12-16 12:00:00
   1pctCO2-bgc
     ###
GISS-E2-1-G
   piControl
     1850-01-16 12:00:00
     2014-12-16 12:00:00
   1pctCO2
     1850-01-16 12:00:00
     2019-12-16 12:00:00
   1pctCO2-rad
 

### Load dictionaries

In [5]:
# Get the last 40 years of the 1% CO2 runs and the corresponding time period in piControl
with open('../pkl_files/period_l40_slice_in_pi.pkl', 'rb') as file:
    period_l40_slice_in_pi = pkl.load(file)
with open('../pkl_files/period_l40_start_year_in_pi.pkl', 'rb') as file:
    period_l40_start_year_in_pi = pkl.load(file)
with open('../pkl_files/period_l40_end_year_in_pi.pkl', 'rb') as file:
    period_l40_end_year_in_pi = pkl.load(file)

with open('../pkl_files/period_l40_slice_in_co2.pkl', 'rb') as file:
    period_l40_slice_in_co2 = pkl.load(file)
with open('../pkl_files/period_l40_start_year_in_co2.pkl', 'rb') as file:
    period_l40_start_year_in_co2 = pkl.load(file)
with open('../pkl_files/period_l40_end_year_in_co2.pkl', 'rb') as file:
    period_l40_end_year_in_co2 = pkl.load(file)

    
# Get each model's variant_id
with open('../pkl_files/variant_id.pkl', 'rb') as file:
    variant_id = pkl.load(file)

# Merge files

```var_table-id_model_case_variant-id_grid_time.nc```

## Helper functions

In [6]:
def writeMergePaths(theseModels, theseCases, theseVariables, tag, toMerge=None):
    if toMerge == True and type(toMerge) == bool:
        filename = 'paths-m'+str(tag)+'.txt'
    if toMerge == False and type(toMerge) == bool:
        filename = 'paths-c'+str(tag)+'.txt'
        
    if type(theseVariables[0]) == list:
        tmp_vars = []
        for item in theseVariables:
            tmp_vars += item
        theseVariables = tmp_vars
    
    outfile = []
    for m in theseModels:
        for c in theseCases:
            for v in theseVariables:
                out = mergetime(m, c, v, toFile=True, toMerge=toMerge, toDelete=False, toPrint=False)
                if type(out) == str:
                    outfile.append(out)
    
    if len(outfile) > 0:
        with open('../bash_scripts/'+filename, 'w') as f:
            f.writelines(outfile)
        print(filename)
    else:
        print('no files')

In [9]:
def mergetime(thisM, thisC, thisV, toPrint=True, toFile=False, toMerge=None, toDelete=False):
    cmipdir = '/tiger/scratch/gpfs/GEOCLIM/bgb2/CMIP/'
    cmipmergedir = '/tiger/scratch/gpfs/GEOCLIM/bgb2/CMIPmerge/'
    
    fsearch = cmipdir+thisM+'/'+thisV+'_'+table_id[thisV]+'_'+thisM+'_'+thisC+'_'+variant_id[thisM]+'_'+grid_label[thisM]+'_*.nc'
    
    files = glob.glob(fsearch)    
    stimes = []
    etimes = []
    for f in files:
        stimes.append(int(f.split('/')[-1].split('_')[-1].split('.')[0].split('-')[0][:4]))
        etimes.append(int(f.split('/')[-1].split('_')[-1].split('.')[0].split('-')[1][:4]))
    
    fname = thisV+'_'+table_id[thisV]+'_'+thisM+'_'+thisC+'_'+variant_id[thisM]+'_'+grid_label[thisM]+'_%s-%s.nc' % (str(min(stimes)).zfill(4)+'01', str(max(etimes)).zfill(4)+'12')
    fdir = cmipmergedir+thisM+'/'+fname
    
    if toPrint:
        print('    ','/'.join(fsearch.split('/')[-2:]))
        if len(stimes) > 1 and len(etimes) > 1:
            print('     - ',fdir)
    
    if toFile and toMerge and type(toMerge) == bool:
        if len(stimes) > 1 and len(etimes) > 1:
            return fsearch+':'+fdir+'\n'
        
    if toFile and not toMerge and type(toMerge) == bool:
        if len(stimes) == 1 and len(etimes) == 1:
            test_fname = fsearch.split('/')[-1][:-4]+'%s-%s.nc' % (str(min(stimes)).zfill(4)+'01', str(max(etimes)).zfill(4)+'12')
            
            if os.path.isfile(cmipdir+thisM+'/'+test_fname) and fname == test_fname:
                return cmipdir+thisM+'/'+fname+':'+cmipmergedir+thisM+'/'+fname+'\n'
            
            else:
                print('ERROR')
                print(fname)
                print(tmp_fname)
                print(cmipdir+thisM+'/'+tmp_fname)
                       
    if toDelete:
        if len(stimes) > 1 and len(etimes) > 1:
            print('----',fdir.split('/')[-1])
            ## os.popen('rm %s' % fdir)

## Create text files with merge/copy paths

In [11]:
## All models with HT + VEG + SW for all cases
# writeMergePaths(models_all, cases, variables, '1', toMerge=True)
# writeMergePaths(models_all, cases, variables, '1', toMerge=False)

## Models with HT + VEG for FULL (1pctCO2) and RAD (1pctCO2-rad)
# writeMergePaths(models_rad, cases_rad, [variables_ht, variables_veg], '2', toMerge=True)
# writeMergePaths(models_rad, cases_rad, [variables_ht, variables_veg], '2', toMerge=False)

## Models with HT + VEG + SW for FULL (1pctCO2) and RAD (1pctCO2-rad)
# writeMergePaths(models_radsw, cases_rad, variables, '3', toMerge=True)
# writeMergePaths(models_radsw, cases_rad, variables, '3', toMerge=False)

## Redo BCC-CSM2-MR
# writeMergePaths(['BCC-CSM2-MR'], cases, variables, '-bcc', toMerge=True)
# writeMergePaths(['BCC-CSM2-MR'], cases, variables, '-bcc', toMerge=False)

**At this point, run the bash scripts ```mergenetcdf.sh paths-mx.txt``` and ```copynetcdf.sh paths-cx.txt```**

## Remove date at end of file name

```var_table-id_model_case_variant-id_grid_time.nc```

In [21]:
mergedtimes = {}

cmipdir = '/tiger/scratch/gpfs/GEOCLIM/bgb2/CMIP/'
cmipmergedir = '/tiger/scratch/gpfs/GEOCLIM/bgb2/CMIPmerge/'

for d in sorted(os.listdir(cmipmergedir)):
    print(d)
    files = sorted(os.listdir(cmipmergedir+d))
    if len(files) > 0:
        for f in files:
            newname = '_'.join(f.split('_')[:-1])+'.nc'
            # print(newname)
            # print('  ',cmipmergedir+d+'/'+f)
            # print('  ',cmipmergedir+d+'/'+newname)
            ## mergedtimes[newname] = f
            ## os.rename(cmipmergedir+d+'/'+f, cmipmergedir+d+'/'+newname)

## with open('../pkl_files/mergedtimes.pkl', 'wb') as file:
##     pkl.dump(mergedtimes, file, pkl.HIGHEST_PROTOCOL)

ACCESS-ESM1-5
BCC-CSM2-MR
CESM2
CMCC-ESM2
CNRM-ESM2-1
CanESM5
EC-Earth3-CC
GFDL-ESM4
GISS-E2-1-G
IPSL-CM6A-LR
MIROC-ES2L
MPI-ESM1-2-LR
MRI-ESM2-0
NorESM2-LM
UKESM1-0-LL


# Create branch time files

In [ ]:
variant_id

{'ACCESS-ESM1-5': 'r1i1p1f1',
 'BCC-CSM2-MR': 'r1i1p1f1',
 'CanESM5': 'r1i1p1f1',
 'CESM2': 'r1i1p1f1',
 'CMCC-ESM2': 'r1i1p1f1',
 'CNRM-ESM2-1': 'r1i1p1f2',
 'EC-Earth3-CC': 'r1i1p1f1',
 'GFDL-ESM4': 'r1i1p1f1',
 'GISS-E2-1-G': 'r101i1p1f1',
 'IPSL-CM6A-LR': 'r1i1p1f1',
 'MIROC-ES2L': 'r1i1p1f2',
 'MPI-ESM1-2-LR': 'r1i1p1f1',
 'MRI-ESM2-0': 'r1i2p1f1',
 'NorESM2-LM': 'r1i1p1f1',
 'UKESM1-0-LL': 'r1i1p1f2'}

# Check merged/copied files

In [37]:
cmiptemp = '/tiger/scratch/gpfs/GEOCLIM/bgb2/CMIPtemp/'
cmip = '/tiger/scratch/gpfs/GEOCLIM/bgb2/CMIP/'

for f in sorted(glob.glob(direct+'*.nc')):
    ncfile = f.split('/')[-1]
    m = ncfile.split('_')[2]
    
    print(m)
    print(cmip+m+'/'+ncfile)
    
    # shutil.move(f, cmip+m+'/'+ncfile)

CMCC-ESM2
/tiger/scratch/gpfs/GEOCLIM/bgb2/CMIP/CMCC-ESM2/hus_Amon_CMCC-ESM2_1pctCO2-bgc_r1i1p1f1_gn_185001-187412.nc
UKESM1-0-LL
/tiger/scratch/gpfs/GEOCLIM/bgb2/CMIP/UKESM1-0-LL/hus_Amon_UKESM1-0-LL_1pctCO2-bgc_r1i1p1f2_gn_185001-194912.nc
UKESM1-0-LL
/tiger/scratch/gpfs/GEOCLIM/bgb2/CMIP/UKESM1-0-LL/hus_Amon_UKESM1-0-LL_1pctCO2-bgc_r1i1p1f2_gn_195001-199912.nc
UKESM1-0-LL
/tiger/scratch/gpfs/GEOCLIM/bgb2/CMIP/UKESM1-0-LL/hus_Amon_UKESM1-0-LL_1pctCO2-rad_r1i1p1f2_gn_185001-194912.nc
UKESM1-0-LL
/tiger/scratch/gpfs/GEOCLIM/bgb2/CMIP/UKESM1-0-LL/hus_Amon_UKESM1-0-LL_1pctCO2-rad_r1i1p1f2_gn_195001-199912.nc
UKESM1-0-LL
/tiger/scratch/gpfs/GEOCLIM/bgb2/CMIP/UKESM1-0-LL/hus_Amon_UKESM1-0-LL_1pctCO2_r1i1p1f2_gn_185001-194912.nc
UKESM1-0-LL
/tiger/scratch/gpfs/GEOCLIM/bgb2/CMIP/UKESM1-0-LL/hus_Amon_UKESM1-0-LL_1pctCO2_r1i1p1f2_gn_195001-199912.nc
UKESM1-0-LL
/tiger/scratch/gpfs/GEOCLIM/bgb2/CMIP/UKESM1-0-LL/hus_Amon_UKESM1-0-LL_piControl_r1i1p1f2_gn_196001-204912.nc
UKESM1-0-LL
/tiger/scrat